# Introduction to connecting and Querying the Augur DB

If you made to this point, welcome! :) This short tutorial will show how to connect to the database and how to do a simple query. If you need the config file please email cdolfi@redhat.com

For Project Sandiego's data, we will be using a personal instance of Augur. Augur is a software suite for collecting and measuring structured data about free and open-source software (FOSS) communities.

Augur gather's trace data for a group of repositories, normalize it into the data model, and provide a variety of metrics about said data. The structure of the data model enables us to synthesize data across various platforms to provide meaningful context for meaningful questions about the way these communities evolve.

All the notebooks in this folder are based on https://github.com/chaoss/augur-community-reports templates. 

## Connect to your database

Until the Operate First enviroment can connect to the DB, use config file to access. Do not push config file to Github repo

In [1]:
import psycopg2
import pandas as pd 
import sqlalchemy as salc
import json
import os

with open("../comm_cage.json") as config_file:
    config = json.load(config_file)

In [2]:
database_connection_string = 'postgresql+psycopg2://{}:{}@{}:{}/{}'.format(config['user'], config['password'], config['host'], config['port'], config['database'])

dbschema='augur_data'
engine = salc.create_engine(
    database_connection_string,
    connect_args={'options': '-csearch_path={}'.format(dbschema)})

### Retrieve Available Respositories

In [3]:
aval_repos = pd.DataFrame()
repo_query = salc.sql.text(f"""
             SET SCHEMA 'augur_data';
             SELECT a.rg_name,
                a.repo_group_id,
                b.repo_name,
                b.repo_id,
                b.forked_from,
                b.repo_archived,
                b.repo_git
            FROM
                repo_groups a,
                repo b
            WHERE
                a.repo_group_id = b.repo_group_id
            ORDER BY
                rg_name,
                repo_name;
    """)
aval_repos = pd.read_sql(repo_query, con=engine)
display(aval_repos)
aval_repos.dtypes

,rg_name,repo_group_id,repo_name,repo_id,forked_from,repo_archived,repo_git
0,3scale,25556,3scale,33134,Parent not available,0.0,https://github.com/3scale/3scale.github.io
1,3scale,25556,3scale-amp-openshift-templates,33126,Parent not available,0.0,https://github.com/3scale/3scale-amp-openshift...
2,3scale,25556,3scale-api-python,33130,3scale-qe/3scale-api-python,0.0,https://github.com/3scale/3scale-api-python
3,3scale,25556,3scale-api-ruby,33158,Parent not available,0.0,https://github.com/3scale/3scale-api-ruby
4,3scale,25556,3scale-go-client,33169,Parent not available,0.0,https://github.com/3scale/3scale-go-client
...,...,...,...,...,...,...,...
10956,zerodayz,25482,sosreport-operator,30872,andreaskaris/sosreport-operator,0.0,https://github.com/zerodayz/sosreport-operator
10957,zerodayz,25482,talos,30863,siderolabs/talos,0.0,https://github.com/zerodayz/talos
10958,zerodayz,25482,tests,30838,kata-containers/tests,0.0,https://github.com/zerodayz/tests
10959,zerodayz,25482,web,30839,openshifttips/web,0.0,https://github.com/zerodayz/web


rg_name           object
repo_group_id      int64
repo_name         object
repo_id            int64
forked_from       object
repo_archived    float64
repo_git          object
dtype: object

### Create a Simpler List for quickly Identifying repo_group_id's and repo_id's for other queries

In [4]:
repolist = pd.DataFrame()

repo_query = salc.sql.text(f"""
             SET SCHEMA 'augur_data';
             SELECT b.repo_id,
                a.repo_group_id,
                b.repo_name,
                a.rg_name,
                b.repo_git
            FROM
                repo_groups a,
                repo b 
            WHERE
                a.repo_group_id = b.repo_group_id 
            ORDER BY
                rg_name,
                repo_name;   

    """)

repolist = pd.read_sql(repo_query, con=engine)

repolist

,repo_id,repo_group_id,repo_name,rg_name,repo_git
0,33134,25556,3scale,3scale,https://github.com/3scale/3scale.github.io
1,33126,25556,3scale-amp-openshift-templates,3scale,https://github.com/3scale/3scale-amp-openshift...
2,33130,25556,3scale-api-python,3scale,https://github.com/3scale/3scale-api-python
3,33158,25556,3scale-api-ruby,3scale,https://github.com/3scale/3scale-api-ruby
4,33169,25556,3scale-go-client,3scale,https://github.com/3scale/3scale-go-client
...,...,...,...,...,...
10956,30872,25482,sosreport-operator,zerodayz,https://github.com/zerodayz/sosreport-operator
10957,30863,25482,talos,zerodayz,https://github.com/zerodayz/talos
10958,30838,25482,tests,zerodayz,https://github.com/zerodayz/tests
10959,30839,25482,web,zerodayz,https://github.com/zerodayz/web


In [5]:
repolist[50:70]

,repo_id,repo_group_id,repo_name,rg_name,repo_git
50,33174,25556,lua-resty-limit-traffic,3scale,https://github.com/3scale/lua-resty-limit-traffic
51,33144,25556,lua-resty-url,3scale,https://github.com/3scale/lua-resty-url
52,33110,25556,lua-rover,3scale,https://github.com/3scale/lua-rover
53,33172,25556,message_bus_client,3scale,https://github.com/3scale/message_bus_client
54,33184,25556,nginx-upstream-fair,3scale,https://github.com/3scale/nginx-upstream-fair
55,33179,25556,node-sass,3scale,https://github.com/3scale/node-sass
56,33154,25556,openid-connect-api-gateway-rhsso,3scale,https://github.com/3scale/openid-connect-api-g...
57,33109,25556,oracle-database,3scale,https://github.com/3scale/oracle-database
58,33143,25556,Penlight-ffi,3scale,https://github.com/3scale/Penlight-ffi
59,33151,25556,pisoni,3scale,https://github.com/3scale/pisoni


### Create a list of all of the tables with the total number of data entries 

In [9]:
data_entries = pd.DataFrame()

repo_query = salc.sql.text(f"""
                ANALYZE;
                SELECT schemaname,relname,n_live_tup 
                  FROM pg_stat_user_tables 
                  ORDER BY n_live_tup DESC;

    """)

data_entries = pd.read_sql(repo_query, con=engine)

data_entries

,schemaname,relname,n_live_tup
0,augur_data,issues,4227162
1,augur_data,pull_request_meta,2629237
2,augur_data,pull_request_labels,1875821
3,augur_data,pull_request_events,1782713
4,augur_data,pull_request_files,1151045
...,...,...,...
435,augur_data,merge_data_0_1103194,0
436,augur_data,merge_data_0_4087352,0
437,augur_data,merge_data_0_3537777,0
438,augur_data,contributors_aliases,0


Congrats you have done your first queries! There will be a few more simple examples below on how to pull an entire table. If you would like to explore on your own, the schema.png on the home sandiego directory will be greatly helpful in your explorations! Happy querying :) 

### Data from the messages 

This data is the collection of all comments from any issue, PR, commit, etc opened. This example shows another side of the database and the types of data we can pull from it. 

In [10]:
mes_data = pd.DataFrame()

repo_query = salc.sql.text(f"""
             SET SCHEMA 'augur_data';
             SELECT * FROM message
    """)

mes_data = pd.read_sql(repo_query, con=engine)

mes_data

,msg_id,rgls_id,platform_msg_id,platform_node_id,repo_id,cntrb_id,msg_text,msg_timestamp,msg_sender_email,msg_header,pltfrm_id,tool_source,tool_version,data_source,data_collection_date
0,25430,None,826722981,MDEyOklzc3VlQ29tbWVudDgyNjcyMjk4MQ==,NaN,25440,I've enabled actions for this repo. LMK if it ...,2021-04-26 10:35:38,None,None,25150,GitHub API Worker,1.0.0,GitHub API,2021-10-21 13:41:55
1,25431,None,826812453,MDEyOklzc3VlQ29tbWVudDgyNjgxMjQ1Mw==,NaN,25449,[APPROVALNOTIFIER] This PR is **NOT APPROVED**...,2021-04-26 12:55:35,None,None,25150,GitHub API Worker,1.0.0,GitHub API,2021-10-21 13:41:55
2,25432,None,826815197,MDEyOklzc3VlQ29tbWVudDgyNjgxNTE5Nw==,NaN,25438,Actions seems to be working now. Thanks 👍,2021-04-26 12:59:50,None,None,25150,GitHub API Worker,1.0.0,GitHub API,2021-10-21 13:41:55
3,25433,None,826817026,MDEyOklzc3VlQ29tbWVudDgyNjgxNzAyNg==,NaN,25440,/cc @oindrillac,2021-04-26 13:02:13,None,None,25150,GitHub API Worker,1.0.0,GitHub API,2021-10-21 13:41:55
4,25434,None,826822231,MDEyOklzc3VlQ29tbWVudDgyNjgyMjIzMQ==,NaN,25449,[APPROVALNOTIFIER] This PR is **NOT APPROVED**...,2021-04-26 13:09:08,None,None,25150,GitHub API Worker,1.0.0,GitHub API,2021-10-21 13:41:55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10009658,14997064,None,885743169,PRRC_kwDOAynl5s40y15B,25729.0,2257510,Works for me locally but may not be supported ...,2022-05-31 14:54:06,None,None,25150,GitHub Pull Request Worker,1.2.0,GitHub API,2022-08-22 11:23:01
10009659,14997065,None,885933776,PRRC_kwDOAynl5s40zkbQ,25729.0,2257510,Not sure what happened with the windows build ...,2022-05-31 17:48:58,None,None,25150,GitHub Pull Request Worker,1.2.0,GitHub API,2022-08-22 11:23:01
10009660,14997069,None,786170310,PRRC_kwDOAynl5s4u3AHG,25729.0,2474932,"I think it's negative for west, since EST is w...",2022-01-17 16:51:30,None,None,25150,GitHub Pull Request Worker,1.2.0,GitHub API,2022-08-22 11:23:01
10009661,14997070,None,786201847,PRRC_kwDOAynl5s4u3Hz3,25729.0,2474932,Is EST being east of UTC a time zone conventio...,2022-01-17 17:28:48,None,None,25150,GitHub Pull Request Worker,1.2.0,GitHub API,2022-08-22 11:23:01


### Contributor affiliation data


This data tells us what is the company affiliation of many open source contributors. This can help tell us which companies are involved in a certian open source project. 

In [8]:
con_aff = pd.DataFrame()

repo_query = salc.sql.text(f"""
             SET SCHEMA 'augur_data';
             SELECT * FROM contributor_affiliations
    """)

con_aff = pd.read_sql(repo_query, con=engine)

con_aff

,ca_id,ca_domain,ca_start_date,ca_last_used,ca_affiliation,ca_active,tool_source,tool_version,data_source,data_collection_date
0,1,samsung.com,1970-01-01,2018-08-01 18:37:54,Samsung,1,load,1.0,load,1970-01-01
1,2,linuxfoundation.org,1970-01-01,2018-08-01 18:37:54,Linux Foundation,1,load,1.0,load,1970-01-01
2,3,ibm.com,1970-01-01,2018-08-01 18:37:54,IBM,1,load,1.0,load,1970-01-01
3,8,walmart.com,1970-01-01,2018-09-01 06:00:00,Walmart,1,load,1.0,load,1970-01-01
4,9,exxonmobil.com,1970-01-01,2018-09-01 06:00:00,Exxon Mobil,1,load,1.0,load,1970-01-01
...,...,...,...,...,...,...,...,...,...,...
515,516,twitter.com,1970-01-01,2018-09-01 06:00:00,Twitter,1,load,1.0,load,1970-01-01
516,517,adobe.com,1970-01-01,2018-09-01 06:00:00,Adobe,1,load,1.0,load,1970-01-01
517,519,acm.org,1970-01-01,2018-09-12 02:01:59,ACM,1,load,1.0,load,1970-01-01
518,520,outdoors@acm.org,1970-01-01,2018-09-12 02:32:53,University of Missouri,1,load,1.0,load,2013-07-15
